# Tensorflow and Pytorch 

## Embedding

In [9]:
import torch
import torch.nn as nn

vocab_size_input = 10
embedding_dim = 20
embedding_matrix_input = torch.rand((10,20))        ## pre-trained embedding matrix of 50 words and 20 dimension
max_sequence_length = 4

embedding_layer = nn.Embedding(num_embeddings = vocab_size_input, embedding_dim = embedding_dim)

embedding_layer.weight = nn.Parameter(data = embedding_matrix_input)       ## this is for pre-trained embedding matrix

embedding_layer.weight.requires_grad = False

encoder_inputs = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8]])

embedded_out = embedding_layer(encoder_inputs)


print("input shape => ", encoder_inputs.shape)
print("embeding matrix shape => ", embedding_matrix_input.shape)
print("embeding weights shape => ", embedding_layer.weight.shape)
print("embeding out shape => ", embedded_out.shape)

input shape =>  torch.Size([2, 4])
embeding matrix shape =>  torch.Size([10, 20])
embeding weights shape =>  torch.Size([10, 20])
embeding out shape =>  torch.Size([2, 4, 20])


In [23]:
from tensorflow.keras.layers import Embedding
import tensorflow as tf

vocab_size_input = 10
embedding_dim = 20
embedding_matrix_input = tf.random.uniform((10,20), minval=0, maxval=1)        ## pre-trained embedding matrix of 50 words and 20 dimension
max_sequence_length = 4

embed_layer = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input], 
                        trainable=False)

input_maatrix = tf.constant([[1, 2, 3, 4], [5, 6, 7, 8]])
embed_out = embed_layer(input_maatrix)

print("input shape => ", input_maatrix.shape)
print("embeding matrix shape => ", embedding_matrix_input.shape)
print("embeding out shape => ", embed_out.shape)
print("embeding weights shape => ", embed_layer.weights)

print(embed_layer.get_weights()[0][2])

print(embedding_matrix_input[2])

input shape =>  (2, 4)
embeding matrix shape =>  (10, 20)
embeding out shape =>  (2, 4, 20)
embeding weights shape =>  [<KerasVariable shape=(10, 20), dtype=float32, path=embedding_12/embeddings>]
[0.31766498 0.6002455  0.23098874 0.8003701  0.22288322 0.76057255
 0.2438178  0.30091774 0.44090402 0.10688007 0.289572   0.8355458
 0.775      0.1713177  0.5051272  0.7596158  0.89606    0.78942907
 0.61408854 0.3347224 ]
tf.Tensor(
[0.31766498 0.6002455  0.23098874 0.8003701  0.22288322 0.76057255
 0.2438178  0.30091774 0.44090402 0.10688007 0.289572   0.8355458
 0.775      0.1713177  0.5051272  0.7596158  0.89606    0.78942907
 0.61408854 0.3347224 ], shape=(20,), dtype=float32)


## Fully Connected

In [35]:
import torch
import torch.nn as nn

# linear_layer = nn.Linear(in_features = 4, out_features = 4)
linear_layer = nn.Linear(in_features = 4, out_features = 6)

# input_matrix = torch.tensor([[1, 2, 4, 5]], dtype=torch.float32)
input_matrix = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8]], dtype=torch.float32)

linear_out = linear_layer(input_matrix)

print("input shape => ", input_matrix.shape)
print("linear shape => ", linear_layer.weight)
print("linear out shape => ", linear_out.shape)

input shape =>  torch.Size([2, 4])
linear shape =>  Parameter containing:
tensor([[ 0.3719,  0.1079,  0.0751,  0.1788],
        [-0.1073,  0.4483, -0.1810,  0.3575],
        [ 0.2456, -0.2400, -0.3230,  0.2244],
        [ 0.1020,  0.2358,  0.1252,  0.4686],
        [-0.3869, -0.2777, -0.1740, -0.3906],
        [-0.3573, -0.2412,  0.0207,  0.4034]], requires_grad=True)
linear out shape =>  torch.Size([2, 6])


In [36]:
from tensorflow.keras.layers import Dense
import tensorflow as tf


dense_layer = Dense(units= 4)
# dense_layer = Dense(units= 6)

input_maatrix = tf.constant([[1, 2, 3, 4], [5, 6, 7, 8]])
# input_maatrix = tf.constant([[1, 2, 3, 4]])
dense_out = dense_layer(input_maatrix)

print("input shape => ", input_maatrix.shape)
print("dense out shape => ", dense_out.shape)
print("dense weights shape => ", dense_layer.weights)

print(dense_layer.get_weights())

input shape =>  (2, 4)
dense out shape =>  (2, 4)
dense weights shape =>  [<KerasVariable shape=(4, 4), dtype=float32, path=dense_10/kernel>, <KerasVariable shape=(4,), dtype=float32, path=dense_10/bias>]
[array([[ 0.73735625, -0.2306118 ,  0.18191022, -0.10457295],
       [ 0.2300995 , -0.3000422 ,  0.15840608,  0.031856  ],
       [-0.2407248 , -0.6900956 ,  0.10056913,  0.4481197 ],
       [-0.4519666 , -0.6903256 ,  0.07607073,  0.75877637]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32)]


In [ ]:


import torch
import torch.nn as nn


class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size        ## let's take glove - 512
        self.heads = heads                  ## 8 head ..... for a perticular word how many meanings you want
        self.head_dim = embed_size // heads     ##  head dim = 64 ... because => 512 // 8

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"      ## if the condition does not match then it will return the error

            ## linear layer does not change shape
        self.values = nn.Linear(embed_size, embed_size)         ## size of input and output is same for Q, K, V 
        self.keys = nn.Linear(embed_size, embed_size)               ## shape => (N, length of ( Q or V or K), embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)         ## fully connected output layer

    def forward(self, values, keys, query, mask):               ## forward function will take some input
        ## mask shape => (N, 1, 1, sen_len)

        ## Get number of training examples
        N = query.shape[0]                              ## total number of training samples
                                                        ## values =>  (N, value_len, embed_size)
                                                        ## keys => (N, key_len, embed_size)
                                                        ## query => (N, query_len, embed_size)

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]


        values = self.values(values)        ## linear layer does not change the shape ... input shape = output shape
        keys = self.keys(keys)
        queries = self.queries(query)

        ## Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)   ## convert the 3D array (N, key_len, embed_size) to 4D array
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)              ## embed_size => n_head * head_dim
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)      ## 512 = 8 * 64

        ## Einsum does matrix mult. for query*keys for each training example
        ## with every other training example, don't be confused by einsum
        ## it's just how I like doing matrix multiplication & bmm

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])       ## einsum => Einstein summation notation
            ## Q * K           ("nqhd,nkhd->nhqk")
            ## (N, q_len, heads, h_dim) * (N, k_len, heads, h_dim) => (N, heads, q_len, k_len)

        ## queries shape: (N, query_len, heads, heads_dim),
        ## keys shape: (N, key_len, heads, heads_dim)
        ## energy: (N, heads, query_len, key_len)

        ## Mask padded indices so their weights become 0 .... for mask attension
        if mask is not None:
            # energy = energy.masked_fill(mask == 0, float("-1e20"))
            energy = energy.masked_fill(mask == 0, float("-inf"))
            # print(energy[0][0][0])
            ## masked_fill(mask, value) replaces all elements in e where the mask is True with the specified value. 
            ## In this case, it replaces those elements with -1e20.

        ## Normalize energy values similarly to seq2seq + attention
        ## so that they sum to 1. Also divide by scaling factor for
        ## better stability
        ## energy and attention dimension is same 
        ## attention is the new weights that can be multiply with values
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)     ## 1/ Dk ** (1/2)  => Dk = dimensionality of K vec
                ## scale the vector and then apply softmax  .. it will give us new weights 
                ## dim=3 means the last last shape of the 4D array 
            ## attention shape: (N, heads, query_len, key_len)

            ## matrix multiplication of attention vector and values vector
            ## (N, heads, query_len, key_len) * (N, value_len, heads, head_dim) = (N, q_len, heads, head_dim)
            ## reshape the 4D vector (N, q_len, heads, head_dim) and create new 3D vector of shape (N, q_len, embd_dim)
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        ## attention shape: (N, heads, query_len, key_len)
        ## values shape: (N, value_len, heads, heads_dim)
        ## out after matrix multiply: (N, query_len, heads, head_dim), then
        ## we reshape and flatten the last two dimensions.

        out = self.fc_out(out)
        ## Linear layer doesn't modify the shape, final shape will be
        ## (N, query_len, embed_size)
        return out



        ## this transformer block can be used is encoder and decoder
        ## this contain ( 1 self attention model => addition and normalization => 1 feed forward layer )
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        """
        Layer normalization is a technique to normalize the input to a neural network layer by adjusting and scaling activations. 
        Unlike batch normalization, which normalizes across the batch dimension, layer normalization normalizes across the feature 
        dimension for each data sample.
        """
        self.norm1 = nn.LayerNorm(normalized_shape = embed_size)       ## Suppose the feature dimension is embed_size 
        self.norm2 = nn.LayerNorm(embed_size)                          ## last dimension of the matrix shape

            ## feed forward ....  input shape and output shape is same
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)     ## (N, query_len, embed_size) this is the output of SelfAttention class

        ## Add skip connection, run through normalization and finally dropout
        ## residual connection with dropouts
        x = self.dropout(self.norm1(attention + query))

        ## feedforward layer of transformer block
        forward = self.feed_forward(x)

        ## residual (skip connection) connection with dropouts
        out = self.dropout(self.norm2(forward + x))
        return out              ## (N, query_len, embed_size)


class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embed_size, num_layers, heads, device,
                forward_expansion, dropout, max_length,):
        super(Encoder, self).__init__()         ## lets take  forward_expansion = 4 .... embed_size = 512, num_layers = 6
        self.embed_size = embed_size            ## heads = 8 , max_length = 100 , src_vocab_size = 10
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)      ## => (N , indices provided, embedding vectors)
                                                                            ##      OR   => (N , src_vocab_size, embedding vectors)
        self.position_embedding = nn.Embedding(max_length, embed_size)

            ## the encoder block will repeat for 6 times ... (num_layers = 6)
            ## 6 encoder is attached one after another
        self.layers = nn.ModuleList(
            [
                TransformerBlock(embed_size, heads, dropout=dropout,forward_expansion=forward_expansion) for _ in range(num_layers)
            ]
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        """
        The expand method works by broadcasting the original tensor to a new shape. Broadcasting is a technique where dimensions of a tensor 
        are automatically expanded to be compatible with other tensors in operations.
        """
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)       ## positions = (N, seq_length)

            ## when we feed a sentence to the encoder this is the first layer where the embedding vector and positional embedding 
            ## vector will come ... addition of both vectors and ont the top of that there is a dropout layer 
            ## before feeding this resultant vector to transformer block (self attention, feed forward, normalization )
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))               ## (N , seq_length, embedding vectors)
        )

        ## In the Encoder the query, key, value are all the same, it's in the
        ## decoder this will change. This might look a bit odd in this case.
        ## the out put of 1st transformer block will be input for the 2nd ..... out put of 2nd transformer block will be input for the 3rd ..
        ## same out put is used as ( Q, K , V) 
        for layer in self.layers:
            out = layer(out, out, out, mask)                ## (N , seq_length, embedding vectors)

        return out


class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()            ## forward_expansion = 4 .... embed_size = 512 .... heads = 8
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):                   ## x => (N, trg_seq_len, embed_size)
        attention = self.attention(x, x, x, trg_mask)                           ## (N, query_len, embed_size)
        query = self.dropout(self.norm(attention + x))                  ## this is the feed forward layer
        out = self.transformer_block(value, key, query, src_mask)           ## (N, trg_seq_length, embedding vectors)
        return out


class Decoder(nn.Module):
    def __init__( self, trg_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length):
        super(Decoder, self).__init__()         ##  trg_vocab_size = 10 ... num_layers = 6 ...  max_length = 100
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)      ## (N , trg_vocab_size, embedding vectors)
        self.position_embedding = nn.Embedding(max_length, embed_size)          ## (N , max_length, embedding vectors)

            ## the decoder block will repeat for 6 times ... (num_layers = 6)
            ## 6 decoder is attached one after another
        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        """
        The expand method works by broadcasting the original tensor to a new shape. Broadcasting is a technique where dimensions of a tensor 
        are automatically expanded to be compatible with other tensors in operations.
        """
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)               ## (N, seq_length)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))         ## (N, trg_seq_length, embed_size)

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)              ## (N, trg_seq_length, embed_size)

        out = self.fc_out(x)                ## fully connected feed forward  =>  (N, trg_seq_length, embed_size)
        return out


class Transformer(nn.Module):
    def __init__( self, src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, embed_size=512, 
                 num_layers=6, forward_expansion=4, heads=8, dropout=0, device="cpu", max_length=100,):
        super(Transformer, self).__init__()

        self.encoder = Encoder( src_vocab_size, embed_size, num_layers, heads, 
                               device, forward_expansion, dropout, max_length,)

        self.decoder = Decoder( trg_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length)

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)      ## (N, 1, 1, src_len)   this will be a true and false matrix
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(N, 1, trg_len, trg_len)
        return trg_mask.to(self.device)


    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out


if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2 ,0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(
        device
    ) 
    trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

    src_pad_idx = 0     ## source padding id
    trg_pad_idx = 0     ## target padding id
    src_vocab_size = 10         ## source vocab size
    trg_vocab_size = 10         ## target vocab size
    model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(
        device
    )
    out = model(x, trg[:, :-1])
    print(out.shape)